# Prelimilaries and Kaggle Data

In [1]:
from IPython.display import clear_output 
!sudo add-apt-repository -y ppa:alessandro-strada/ppa
!sudo apt update && sudo apt install google-drive-ocamlfuse
!google-drive-ocamlfuse

Hit:1 http://us-east-2.ec2.archive.ubuntu.com/ubuntu bionic InRelease
Hit:2 http://us-east-2.ec2.archive.ubuntu.com/ubuntu bionic-updates InRelease  
Hit:3 http://us-east-2.ec2.archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:4 http://security.ubuntu.com/ubuntu bionic-security InRelease              
Get:5 https://nvidia.github.io/libnvidia-container/stable/ubuntu18.04/amd64  InRelease [1484 B]
Get:6 https://nvidia.github.io/nvidia-container-runtime/stable/ubuntu18.04/amd64  InRelease [1481 B]
Get:7 https://nvidia.github.io/nvidia-docker/ubuntu18.04/amd64  InRelease [1474 B]
Hit:8 https://apt.repos.neuron.amazonaws.com bionic InRelease                  
Get:9 http://ppa.launchpad.net/alessandro-strada/ppa/ubuntu bionic InRelease [15.4 kB]
Hit:10 http://ppa.launchpad.net/openjdk-r/ppa/ubuntu bionic InRelease         
Get:11 http://ppa.launchpad.net/alessandro-strada/ppa/ubuntu bionic/main amd64 Packages [1368 B]
Get:12 http://ppa.launchpad.net/alessandro-strada/ppa/ubuntu bionic

In [2]:
!sudo apt-get install w3m -y # to act as web browser 
!sudo apt-get install -y xdg-utils
!xdg-settings set default-web-browser w3m.desktop # to set default browser 


! google-drive-ocamlfuse /content/cmudrive
! pip install kaggle wandb torch-summary
! mkdir ~/.kaggle
! cp /content/cmudrive/IDL/kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
! pip install --upgrade --force-reinstall --no-deps kaggle 
! kaggle config set -n path -v /content

! wandb login 4bdbe9c204105e1264fe3f54df2732fd1fff8040

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following packages were automatically installed and are no longer required:
  libaio1 librados2 librbd1
Use 'sudo apt autoremove' to remove them.
The following additional packages will be installed:
  libgc1c2
Suggested packages:
  cmigemo dict dict-wn dictd libsixel-bin mpv w3m-el w3m-img xdg-utils xsel
The following NEW packages will be installed:
  libgc1c2 w3m
0 upgraded, 2 newly installed, 0 to remove and 8 not upgraded.
Need to get 1006 kB of archives.
After this operation, 2939 kB of additional disk space will be used.
Get:1 http://us-east-2.ec2.archive.ubuntu.com/ubuntu bionic/main amd64 libgc1c2 amd64 1:7.4.2-8ubuntu1 [81.8 kB]
Get:2 http://us-east-2.ec2.archive.ubuntu.com/ubuntu bionic/main amd64 w3m amd64 0.5.3-36build1 [924 kB]
Fetched 1006 kB in 0s (44.3 MB/s)
Selecting previously unselected package libgc1c2:amd64.
(Reading database ... 142950 files and directories currently

In [3]:
!pip install python-Levenshtein
!pip install torchsummaryX # We also install a summary package to check our model's forward before training
# !git clone --recursive https://github.com/parlance/ctcdecode.git
!pip install wget
!pip install adamp
# %cd ctcdecode
# !pip install .
# %cd ..

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.5/50.5 KB 3.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for python-Levenshtein: filename=python_Levenshtein-0.12.2-cp38-cp38-linux_x86_64.whl size=82258 sha256=f91a68b80fa3a78a86d78345b7fc93fe583a650a3af1efc940f5612160a40afc
  Stored in directory: /home/ubuntu/.cache/pip/wheels/d7/0c/76/042b46eb0df65c3ccd0338f791210c55ab79d209bcc269e2c7
Successfully built python-Levenshtein
Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
  Preparing metadata (setup.py) ... done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9672 sha256=1a5bffe8903c8845f38e58db4c5521df8552dc774dd9aa47538773c72d7ce046
  Stored in directory: /home/ubuntu/.cache/pip/wheels/bd/a8/c3/3cf2c14a1837a4e04bd98631724e81f33f462d86a1d

In [4]:
! kaggle competitions download -c 11-785-s22-hw3p2

! unzip -q /content/competitions/11-785-s22-hw3p2/11-785-s22-hw3p2.zip -d /content
! mv /content/hw3p2_student_data/hw3p2_student_data /content/speech_data
! rm -rf /content/hw3p2_student_data

100%|█████████████████████████████████████▉| 1.84G/1.84G [00:24<00:00, 86.4MB/s]
100%|██████████████████████████████████████| 1.84G/1.84G [00:24<00:00, 79.3MB/s]


# Dependencies

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchaudio.transforms as taf
from torchsummaryX import summary
# from torchsummary import summary

from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence, pad_packed_sequence
from torch.utils.data import Dataset, DataLoader
from torch.cuda.amp import GradScaler, autocast
from torch.optim.lr_scheduler import ExponentialLR, ReduceLROnPlateau

from sklearn.metrics import accuracy_score
import gc
import zipfile
import pandas as pd
import numpy as np
from tqdm import tqdm
import os
import csv
import yaml
import wandb
import datetime
import time

# imports for decoding and distance calculation
# import ctcdecode
import Levenshtein
# from ctcdecode import CTCBeamDecoder
from phonemes import PHONEME_MAP, PHONEMES
from adamp import AdamP

torch.autograd.set_detect_anomaly(False)
torch.autograd.profiler.profile(False)
torch.autograd.profiler.emit_nvtx(False)

import warnings
import multiprocessing
warnings.filterwarnings('ignore')

DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
print("Device: ", DEVICE)

def header(head):
    print("-"*80)
    print(f"\t\t\t\t{head.upper()}")
    print("-"*80)

Device:  cuda


# Dataset and dataloading

In [2]:
class LibriSamples(torch.utils.data.Dataset):
    def __init__(self, data_path=r"/content/speech_data", partition="train", transforms=None): # You can use partition to specify train or dev
        partition_path = os.path.join(data_path, partition)
        self.transforms = transforms

        self.test = True if partition == "test" else False

        self.X_dir = os.path.join(partition_path, 'mfcc')
        self.Y_dir = os.path.join(partition_path, 'transcript')

        if self.test:
            with open(os.path.join(partition_path, 'test_order.csv'),"r") as f:
                self.X_files = list(csv.reader(f))[1:]
            self.X_data = [np.load(os.path.join(self.X_dir, xfile[0])) for xfile in tqdm(self.X_files, desc="Data", position=0, leave=True)]
        else:
            self.X_files = os.listdir(self.X_dir)
            self.Y_files = os.listdir(self.Y_dir)

            self.X_data = [np.load(os.path.join(self.X_dir, xfile)) for xfile in tqdm(self.X_files, desc="Data", position=0, leave=True)]
            self.Y_data = [np.load(os.path.join(self.Y_dir, yfile)) for yfile in tqdm(self.Y_files, desc="Labels", position=0, leave=True)]

            assert(len(self.X_data) == len(self.Y_data))

        self.phonemes = PHONEMES
        self.phonemes_idx = dict(zip(PHONEMES, range(len(PHONEMES))))

    def __len__(self):
        return len(self.X_files)

    def __getitem__(self, idx):
        X = self.X_data[idx]
        X = (X - X.mean(axis=0))/X.std(axis=0)
        if self.test:
            return torch.from_numpy(X)
        else:
            Y = self.Y_data[idx][1:-1] 
            Yy = np.array([torch.tensor(self.phonemes_idx[t]) for t in Y])
            return torch.from_numpy(X), torch.from_numpy(Yy)
    
    def collate_fn(self, batch):
        if self.test:
            batch_x = [x for x in batch]
            batch_x_pad = pad_sequence(batch_x, batch_first=True)
            lengths_x = [len(x) for x in batch_x]
            return batch_x_pad, torch.tensor(lengths_x)
        else:
            batch_x = [x for x,_ in batch]
            batch_y = [y for _,y in batch]
            batch_x_pad = pad_sequence(batch_x, batch_first=True)
            batch_y_pad = pad_sequence(batch_y, batch_first=True)
            lengths_x = [len(x) for x in batch_x]
            lengths_y = [len(y) for y in batch_y]
            return batch_x_pad, batch_y_pad, torch.tensor(lengths_x), torch.tensor(lengths_y)

In [3]:
# batch_size = 64
# train_data = LibriSamples(partition='train')
# # val_data = LibriSamples(partition='dev')
# # test_data = LibriSamples(partition='test')

# train_loader = DataLoader(train_data, batch_size=128, shuffle=True, drop_last=True, collate_fn = train_data.collate_fn, num_workers=2)
# # val_loader = DataLoader(val_data, batch_size=128, shuffle=False, drop_last=True, collate_fn = val_data.collate_fn, num_workers=2)
# # test_loader = DataLoader(test_data, batch_size=128, shuffle=False, drop_last=False, collate_fn = test_data.collate_fn)

# print("Batch size: ", batch_size)
# print("Train dataset samples = {}, batches = {}".format(train_data.__len__(), len(train_loader)))
# # print("Val dataset samples = {}, batches = {}".format(val_data.__len__(), len(val_loader)))
# # print("Test dataset samples = {}, batches = {}".format(test_data.__len__(), len(test_loader)))

# # Test code for checking shapes and return arguments of the train and val loaders

In [4]:
# for data in train_loader:
#     x, y, lx, ly = data # if you face an error saying "Cannot unpack", then you are not passing the collate_fn argument
#     print(x.shape, y.shape, lx.shape, ly.shape)
#     print(x[0], lx[0])
#     print(y[0], ly[0])
#     break

# Model

### ResNet

In [5]:
# from torch.nn.modules.conv import Conv1d
# torch.cuda.empty_cache()
# class Block(nn.Module):
#     def __init__(self, in_channels, out_channels, stride=1, identity=None):
#         super(Block, self).__init__()
#         self.expansion = 2

#         self.conv1 = nn.Conv1d(in_channels, out_channels, kernel_size=1, stride=1, padding=0)
#         self.bn1 = nn.BatchNorm1d(out_channels)
#         self.conv2 = nn.Conv1d(out_channels, out_channels, kernel_size=3, stride=stride, padding=1)
#         self.bn2 = nn.BatchNorm1d(out_channels)
#         self.conv3 = nn.Conv1d(out_channels, out_channels * self.expansion, kernel_size=1, stride=1, padding=0)
#         self.bn3 = nn.BatchNorm1d(out_channels * self.expansion)
#         self.relu = nn.ReLU()

#         self.identity_downsample = nn.Sequential(
#                 nn.Conv1d(in_channels, out_channels * self.expansion, kernel_size=1, stride=stride),
#                 nn.BatchNorm1d(out_channels * self.expansion)
#             )

#     def forward(self, x):
#         identity = x
#         identity = self.identity_downsample(identity)

#         x = self.conv1(x)
#         x = self.bn1(x)
#         x = self.relu(x)
#         x = self.conv2(x)
#         x = self.bn2(x)
#         x = self.relu(x)
#         x = self.conv3(x)
#         x = self.bn3(x)
        
#         x += identity
#         x = self.relu(x)
#         return x

# class ResNet(nn.Module): # [3, 4, 6]
#     def __init__(self):
#         super(ResNet, self).__init__()
#         self.in_channels = 13
#         self.out_channels = 64
#         self.stem = nn.Sequential(
#             nn.Conv1d(self.in_channels, 64, kernel_size=3),
#             nn.BatchNorm1d(64),
#             nn.ReLU(),
#         )

#         self.in_channels = 64

#         self.resnet1 = self._make_layers(num_blocks=1, out_channels=64, stride=1)
#         self.resnet2 = self._make_layers(num_blocks=1, out_channels=128, stride=2)
#         self.resnet3 = self._make_layers(num_blocks=1, out_channels=256, stride=2)

#     def _make_layers(self, num_blocks, out_channels, stride):
#         identity_down = None
#         layers = []
#         if stride != 1 or self.in_channels != out_channels * 2:
#             identity_down = nn.Sequential(
#                 nn.Conv1d(self.in_channels, out_channels * 2, kernel_size=1),
#                 nn.BatchNorm1d(out_channels * 2)
#             )

#         layers.append(Block(self.in_channels, out_channels, stride, identity_down))
#         self.in_channels = out_channels*2

#         for i in range(num_blocks - 1):
#             layers.append(Block(self.in_channels, out_channels))

#         return nn.Sequential(*layers)

#     def forward(self, x):
#         x = self.stem(x)
#         x = self.resnet1(x)
#         x = self.resnet2(x)
#         x = self.resnet3(x)

#         return x 

### Arch

In [6]:
# class SpeechNet(nn.Module):
#     def __init__(self, config): # You can add any extra arguments as you wish
#         super().__init__()
#         lstm_params = config['arch']
#         vocab_size = len(PHONEMES)

#         self.stride = 1

#         # Embedding
#         embeddings = [13] + lstm_params['embedding_size']
#         embedding_layers = []
#         for i in range(len(embeddings) - 1):
#             embedding_layers.append(nn.Conv1d(embeddings[i], embeddings[i+1], kernel_size=3, padding=1))
#             embedding_layers.append(nn.BatchNorm1d(embeddings[i+1]))
#             embedding_layers.append(nn.ReLU())
#             # Maybe add a Pooling
#         self.embeddings = nn.Sequential(*embedding_layers)
#         embedding_size = embeddings[-1]

#         # Block Embedding
#         # self.embeddings = nn.Sequential(
#         #     # nn.Conv1d(13, 64, kernel_size=5, stride=2, padding=2),
#         #     # nn.BatchNorm1d(64),
#         #     # nn.ReLU(),
#         #     # nn.MaxPool1d(kernel_size=3, stride=1, padding=1),
#         #     nn.Conv1d(64, 64, kernel_size=1, stride=1, padding=0),
#         #     nn.BatchNorm1d(64),
#         #     nn.Conv1d(64, 64, kernel_size=3, stride=2, padding=1),
#         #     nn.BatchNorm1d(64),
#         #     nn.Conv1d(64, 256, kernel_size=1, stride=1, padding=0),
#         #     nn.BatchNorm1d(256),
#         #     nn.ReLU()
#         # )
#         # embedding_size = 256

#         # ResNet Embedding
#         # self.embeddings = ResNet(layers=[1])
#         # embedding_size = 256

#         # RNN Block
#         if config['rnn'] == 'GRU':
#             self.lstm = nn.GRU(
#                 input_size=embedding_size,
#                 hidden_size=lstm_params['hidden_size'],
#                 num_layers=lstm_params['num_layers'],
#                 dropout = lstm_params['dropout'],
#                 bidirectional=True,
#                 batch_first=True
#             )
#         else:
#             self.lstm = nn.LSTM(
#                 input_size=embedding_size,
#                 hidden_size=lstm_params['hidden_size'],
#                 num_layers=lstm_params['num_layers'],
#                 dropout = lstm_params['dropout'],
#                 bidirectional=True,
#                 batch_first=True
#                 )

#         # Classification
#         self.classification = nn.Sequential(
#             nn.Linear(lstm_params['hidden_size'] * 2, 2048),
#             nn.Dropout(p=0.2),
#             nn.Linear(2048, 1024),
#             nn.Dropout(p=0.2),
#             nn.Linear(1024, vocab_size)
#         )

#     def forward(self, x, x_len): 
#         x = torch.transpose(self.embeddings(torch.transpose(x, 1, 2)), 1, 2)
#         # x_len = torch.clamp(x_len, min=0, max=x.shape[1])
#         packed_input = pack_padded_sequence(x, x_len, enforce_sorted=False, batch_first=True)
#         out, _ = self.lstm(packed_input)
#         out, lengths  = pad_packed_sequence(out, batch_first=True)
#         out = self.classification(out)
#         out = F.log_softmax(out, dim=2)
#         return out, lengths

#     def initialize_weights(self):
#         for m in self.modules():
#             if isinstance(m, nn.Conv1d):
#                 nn.init.trunc_normal_(m.weight, std=0.2)
#             elif isinstance(m, nn.BatchNorm2d):
#                 nn.init.constant_(m.weight, 1)
#                 nn.init.constant_(m.bias, 0)
#             elif isinstance(m, nn.Linear):
#                 nn.init.trunc_normal_(m.weight, std=0.2)
#                 nn.init.constant_(m.bias, 0)

In [7]:
# config = {
#     '': 'LSTM-4layer-medium',
#     'batch_size': 128,
#     'epochs': 100,
#     'scheduler': 'CALR',             # CosineAnnealingLR (CALR), ReduceLRonPlateau (RLRP)
#     'optimizer': 'AdamP',            # SGD, Adam, AdamW, AdamP
#     'rnn': 'LSTM',                   # GRU, LSTM
#     'optim': {'lr': 0.002},
#     'decoder': {'beam_width': 3, 'cutoff_top_n': 40, 'cutoff_prob': 1.0},
#     'arch': {'embedding_size': [64, 256], 'hidden_size': 256, 'num_layers': 4, 'dropout': 0.25},
#     'save': True,
#     'log': True,
#     'randomize': False,
# }
# model = SpeechNet(config).cuda()
# print(model)
# # out, out_lengths = model(x.cuda(), lx)
# # print(out.shape, out_lengths.shape)
# # print(y.shape, ly.shape)
# summary(model, x.cuda(), lx)

### ASRNet

In [8]:
class SpeechNet(nn.Module):
    def __init__(self, config): # You can add any extra arguments as you wish
        super().__init__()
        lstm_params = config['arch']
        vocab_size = len(PHONEMES)

        self.stride = 1

        # Embedding
        # self.skip = lstm_params['skip_connect']
        # embeddings = lstm_params['embedding_size']
        # embedding_layers = []
        # for i in range(len(embeddings) - 1):
        #     conv_params = lstm_params['embedding_args'][i]
        #     embedding_layers.append(nn.Conv1d(embeddings[i], embeddings[i+1], **conv_params))
        #     embedding_layers.append(nn.BatchNorm1d(embeddings[i+1]))
        #     embedding_layers.append(nn.Dropout(0.15))
        #     if self.skip:
        #         if i != len(embeddings) - 2:
        #             embedding_layers.append(nn.ReLU())
        #     else:
        #         embedding_layers.append(nn.ReLU())
        #     # Maybe add a Pooling
        # self.embeddings = nn.Sequential(*embedding_layers)
        # embedding_size = embeddings[-1]

        # if self.skip:
        #     self.residual = nn.Sequential(
        #         nn.Conv1d(embeddings[0], embeddings[-1], **lstm_params['embedding_args'][1]),
        #         nn.BatchNorm1d(embeddings[-1])
        #     )
        #     self.relu = nn.ReLU()

        # Block Embedding
        self.embeddings = nn.Sequential(
            nn.Conv1d(13, 128, kernel_size=5, stride=2, padding=2),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.Conv1d(128, 256, kernel_size=3, stride=1, padding='same'),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Conv1d(256, 512, kernel_size=3, stride=1, padding='same'),
            nn.BatchNorm1d(512),
            nn.ReLU(),
        )
        embedding_size = 512

        # ResNet Embedding
        # self.embeddings = ResNet().cuda()
        # embedding_size = 512

        # RNN Block
        if config['rnn'] == 'GRU':
            self.lstm = nn.GRU(
                input_size=embedding_size,
                hidden_size=lstm_params['hidden_size'],
                num_layers=lstm_params['num_layers'],
                dropout = lstm_params['dropout'],
                bidirectional=True,
                batch_first=True
            )
        else:
            self.lstm = nn.LSTM(
                input_size=embedding_size,
                hidden_size=lstm_params['hidden_size'],
                num_layers=lstm_params['num_layers'],
                dropout = lstm_params['dropout'],
                bidirectional=True,
                batch_first=True
                )

        # Classification
        self.classification = nn.Sequential(
            nn.Linear(lstm_params['hidden_size'] * 2, 2048),
            nn.Dropout(p=0.35),
            nn.ReLU(),
            nn.Linear(2048, vocab_size)
        )

    def forward(self, x, x_len): 
        x = torch.transpose(x, 1, 2)
        # if self.skip:
        #     residual = self.residual(x)
        x = self.embeddings(x)
        # if self.skip:
        #     x = x + residual
        #     x = self.relu(x)
            
        x = torch.transpose(x, 1, 2)
#         x_len = torch.clamp(x_len, min=0, max=x.shape[1])
        x_len = x_len // 2

        packed_input = pack_padded_sequence(x, x_len, enforce_sorted=False, batch_first=True)
        out, _ = self.lstm(packed_input)
        out, lengths  = pad_packed_sequence(out, batch_first=True)
        out = self.classification(out)
        out = F.log_softmax(out, dim=2)
        return out, lengths

    def initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv1d):
                nn.init.trunc_normal_(m.weight, std=0.2)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.trunc_normal_(m.weight, std=0.2)
                nn.init.constant_(m.bias, 0)

In [9]:
# torch.cuda.empty_cache()
# config = {
#     '': 'LSTM-4layer-medium',
#     'batch_size': 128,
#     'epochs': 100,
#     'scheduler': 'CALR',             # CosineAnnealingLR (CALR), ReduceLRonPlateau (RLRP)
#     'optimizer': 'AdamP',            # SGD, Adam, AdamW, AdamP
#     'rnn': 'LSTM',                   # GRU, LSTM
#     'optim': {'lr': 0.002},
#     'decoder': {'beam_width': 3, 'cutoff_top_n': 40, 'cutoff_prob': 1.0},
#     'arch': {'embedding_size': [64, 256], 'hidden_size': 512, 'num_layers': 4, 'dropout': 0.25},
#     'save': True,
#     'log': True,
#     'randomize': False,
# }
# model = SpeechNet(config).cuda()
# # print(model)
# # out, out_lengths = model(x.cuda(), lx)
# # print(out.shape, out_lengths.shape)
# # print(y.shape, ly.shape)
# summary(model, x.cuda(), lx)

# Training Configuration

In [10]:
def convert_to_string(tokens, seq_len, phoneme_map, phonemes):
    return "".join([phoneme_map[phonemes[x]] for x in tokens[0:seq_len]])

def calculate_levenshtein(decoder, h, y, lh, ly):
    phoneme_map = dict(zip(PHONEMES, PHONEME_MAP))
    batch_size = ly.shape[0]
    dist = 0
    beam_results, beam_scores, timesteps, out_lens = decoder.decode(h, seq_lens = lh)

    for i in range(batch_size): # Loop through each element in the batch
        h_string = convert_to_string(beam_results[i][0], out_lens[i][0], phoneme_map, PHONEMES)
        y_string = convert_to_string(y[i], ly[i], phoneme_map, PHONEMES)
        dist += Levenshtein.distance(h_string, y_string)
    dist/=batch_size
    return dist

In [11]:
class ModelSetup:
    def __init__(self, config, save_path):
        self.config = config
        self.log = config['log']
        self.save = config['save']
        print(f"Saving : {self.save} and Logging : {self.log}")

        self.phoneme_map = dict(zip(PHONEMES, PHONEME_MAP))
        self.phonemes_ctc = PHONEME_MAP

        self.SAVE_DIR = save_path
        self.DATA_DIR = r"/content/speech_data" 

    def __gen_model_name(self):
        # Generate a model name based on config
        save_name = ''

        for key, val in self.config.items():
            abbr = key[0] if len(key) > 2 else key
            if isinstance(val, dict):
                data = 'lr' + str(val["lr"])
                save_name += data
                break
            elif key == '':
                save_name += abbr + str(val)
                for key, val in self.config['arch'].items():
                    save_name += '-' + str(val)
                save_name += '_'
            else:
                data = abbr + str(val) + '_'
                save_name += data
                

        if self.config['randomize']:
            save_name = save_name + "-v" + str(np.random.randint(10, 1000))
        print("\nModel Name: ", save_name)
        self.model_name = save_name

    def __save_model_params(self, continue_train):
        # Create Model Directory
        save_path = os.path.join(self.SAVE_DIR, self.model_name)
        if not continue_train:
            try:
                os.mkdir(save_path)
            except FileExistsError:
                d = input("Model name already exists. Delete existing model? (y/n)")
                if d == 'y':
                    import shutil
                    shutil.rmtree(save_path)
                    os.mkdir(save_path)
                else:
                    print("Exiting!")
                    exit(0)
                    return None

            os.mkdir(os.path.join(save_path, 'Checkpoints'))
            # Saving Model Configuration
            with open(os.path.join(save_path, 'model_config.yaml'), 'w') as metadata:
                yaml.dump({'Experiment': self.config['']}, metadata, indent=4, default_flow_style=False)
                yaml.dump(self.config, metadata, indent=4, default_flow_style=False)
            print("Model to be saved at: ", save_path)
        self.model_path = save_path

    def dataloaders(self): 
        # self.train_transforms = [
        #                          taf.TimeMasking(10, p=0.25),
        #                          taf.FrequencyMasking(1)

        # ]
        # self.val_transforms = []
        self.train_data = LibriSamples(partition='train')
        self.val_data = LibriSamples(partition='dev')

        self.train_loader = DataLoader(
            self.train_data, 
            batch_size=self.config['batch_size'], 
            shuffle=True, 
            drop_last=True, 
            collate_fn = self.train_data.collate_fn, 
            num_workers=4,
            pin_memory=True) 
        self.val_loader = DataLoader(
            self.val_data, 
            batch_size=self.config['batch_size'], 
            shuffle=False, 
            drop_last=True, 
            collate_fn = self.val_data.collate_fn, 
            num_workers=4,
            pin_memory=True)

    def save_checkpoint(self, epoch, model, optimizer, loss):
        print("Saving Checkpoint!")
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': loss
            }, os.path.join(self.model_path, 'Checkpoints', 'chkpt_' + str(epoch) + '.pth'))

    def save_model(self, epoch=None, onnx=False):
        print("Saving Model!")
        try:
            if not self.save:
                self.__save_model_params()
            if epoch is None:
                name = os.path.join(self.model_path, "model_" + str(epoch) + ".pth")
            else:
                name = os.path.join(self.model_path, "model" + ".pth")
            torch.save(self.model.state_dict(), name)
            if onnx:
                torch.onnx.export(self.model, name.split('.')[0] + '.onnx')
                wandb.save(name.split('.')[0] + '.onnx')
        except:
            print("Model couldn't be saved!")

    def setup(self, continue_train=False, chkpt=None):
        header("Model Setup")

        # Model
        self.model = SpeechNet(self.config).cuda()
        summary(self.model, torch.randn(128, 1692, 13).cuda(), torch.tensor([128]))
        self.__gen_model_name()
        if self.save: self.__save_model_params(continue_train)

        # Loss
        self.criterion = nn.CTCLoss()

        # Decoder
#         self.decoder = CTCBeamDecoder(
#                 self.phonemes_ctc,
#                 blank_id=0,
#                 log_probs_input=True,
#                 **self.config['decoder']
#         )
        
        # Optimizer
        if self.config["optimizer"] == 'SGD':
            self.optimizer = optim.SGD(self.model.parameters(), **self.config['optim'])
        elif self.config["optimizer"] == "Adam":
            self.optimizer = optim.Adam(self.model.parameters(), lr=0.001)
        elif self.config["optimizer"] == "AdamW":
            self.optimizer = optim.AdamW(self.model.parameters(), **self.config['optim'])
        elif self.config["optimizer"] == "AdamP":
            self.optimizer = AdamP(self.model.parameters(), **self.config['optim'])

        self.chkpt = 0
        if continue_train:
            self.chkpt = chkpt
            assert chkpt is not None

            chkpt_path = os.path.join(self.model_path, 'Checkpoints', 'chkpt_' + str(chkpt) + '.pth')
            try:
                checkpoint = torch.load(chkpt_path)
            except FileNotFoundError:
                print("Checkpoint not found in the directory!")
                print("Incorrect: ", chkpt_path)
                exit(0)

            self.model.load_state_dict(checkpoint['model_state_dict'])
            self.optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

        # Scheduler
        if self.config["scheduler"] == 'CALR':
            self.scheduler = optim.lr_scheduler.CosineAnnealingLR(self.optimizer, T_max=(len(self.train_loader) * self.config['epochs']))
        elif self.config["scheduler"] == 'RLRP':
            self.scheduler = optim.lr_scheduler.ReduceLROnPlateau(
                self.optimizer, 
                mode='min', 
                factor=0.5, 
                patience=10,
                threshold=0.25,
                cooldown=15)
        elif self.config["scheduler"] == None:
            pass

        self.scaler = torch.cuda.amp.GradScaler()
    
    def train(self, continue_train=False, save_freq=2):
        header("Training")
        epochs = self.config['epochs']
        batch_size = self.config['batch_size']

        if self.log:
            wandb.init(project="hw3-chinmay", entity="dl-study-group", config=self.config, name=self.config[''])
            wandb.watch(self.model, criterion=self.criterion, log="all", log_freq=batch_size, idx=None)

        delta_time = datetime.timedelta(seconds = 0)
        for epoch in range(self.chkpt, epochs + 50):
            start_time = time.time()

            self.model.train()
            print("\nEpoch-", epoch + 1)
            batch_bar = tqdm(total=len(self.train_loader), dynamic_ncols=True, leave=False, position=0, desc='Train') 
            total_loss = 0

            for i, data in enumerate(self.train_loader, 0):
                torch.cuda.empty_cache()
                self.optimizer.zero_grad()

                x, y, x_len, y_len = data
                x = x.cuda()
                y = y.cuda()

                with torch.cuda.amp.autocast():    
                    output, output_len = self.model(x, x_len)    # B x T x 41
                    output_transposed = torch.transpose(output, 0, 1)              # T x B x 41
                    loss = self.criterion(output_transposed, y, output_len, y_len)

                total_loss += loss.item()
                batch_bar.set_postfix(
                    loss="{:.04f}".format(float(total_loss / (i + 1))),
                    lr="{:.04f}".format(float(self.optimizer.param_groups[0]['lr']))
                    )
                self.scaler.scale(loss).backward() 
                self.scaler.step(self.optimizer)
                self.scaler.update()
                if self.config["scheduler"] == 'CALR': self.scheduler.step()
                batch_bar.update()
  
            batch_bar.close()
            trainlos = float(total_loss / len(self.train_loader))
            trainlra = float(self.optimizer.param_groups[0]['lr'])
            print(f"Epoch {epoch + 1}/{epochs} | Train Loss {trainlos:.04f} | Learning Rate {trainlra:.04f}")

            if self.log:
                    wandb.log({"Training Loss": trainlos, "Learning Rate": trainlra})
                    
#             lev = self.validate()
            if self.config["scheduler"] == 'RLRP': self.scheduler.step(trainlos)

            delta_time += datetime.timedelta(seconds = (time.time() - start_time))
            time_lapsed = delta_time
            time_left = delta_time * (epochs - epoch - 1) / (epoch + 1)
            print(f"Time lapsed = {str(time_lapsed)}")
            print(f"Time left = {str(time_left)}")

            if self.save:
                # Save Model
                if epoch % save_freq == 0: self.save_model(epoch)
                # Save Checkpoint
                self.save_checkpoint(epoch, self.model, self.optimizer, total_loss / len(self.train_loader))
        
    def validate(self):
        self.model.eval()
        batch_bar = tqdm(total=len(self.val_loader), position=0, leave=False, desc='Val')
        total_levenshtein = 0
        for i, data in enumerate(self.val_loader, 0):
            x, y, x_len, y_len = data
            x = x.cuda()
            y = y.cuda()

            with torch.no_grad():    
                output, output_len = self.model(x, x_len)

            total_levenshtein += calculate_levenshtein(self.decoder, output, y, output_len, y_len)
            
            batch_bar.set_postfix(LD="{:.04f}".format(float(total_levenshtein / (i + 1))))
            batch_bar.update()

        batch_bar.close()
        val_lev = float(total_levenshtein / len(self.val_loader))
        print("\nValidation LD: {:.04f}".format(val_lev))
        if self.log:
            wandb.log({"Validation LD": val_lev})

        return val_lev

# Hyperparameters and Run

In [12]:
config = {
    '': 'AWS_L4600-Conv3-L2-Drop6',
    'batch_size': 128,
    'epochs': 105,
    'scheduler': 'CALR',             # CosineAnnealingLR (CALR), ReduceLRonPlateau (RLRP)
    'optimizer': 'AdamP',            # SGD, Adam, AdamW, AdamP
    'rnn': 'LSTM',                   # GRU, LSTM
    'optim': {'lr': 0.002},
    'decoder': {'beam_width': 3, 'cutoff_top_n': 40, 'cutoff_prob': 1.0},
    'arch': {'hidden_size': 600, 'num_layers': 4, 'dropout':0.6},
    'save': True,
    'log': True,
    'randomize': False,
}

# SpeechNet
folder_path = r'/content/cmudrive/IDL/hw3-last'
asr = ModelSetup(config, save_path = folder_path)
asr.dataloaders()

# # Continue Training
# torch.cuda.empty_cache()
# asr.setup(continue_train=True, chkpt=58)
# asr.train(continue_train=True)
# asr.save_model()
# if asr.log: wandb.finish()
# latest_model_name = asr.model_name

Saving : True and Logging : True


Labels: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2703/2703 [00:00<00:00, 3640.73it/s]


In [13]:
torch.cuda.empty_cache()
asr.setup()

--------------------------------------------------------------------------------
				MODEL SETUP
--------------------------------------------------------------------------------
                              Kernel Shape     Output Shape     Params  \
Layer                                                                    
0_embeddings.Conv1d_0         [13, 128, 5]  [128, 128, 846]     8.448k   
1_embeddings.BatchNorm1d_1           [128]  [128, 128, 846]      256.0   
2_embeddings.ReLU_2                      -  [128, 128, 846]          -   
3_embeddings.Conv1d_3        [128, 256, 3]  [128, 256, 846]     98.56k   
4_embeddings.BatchNorm1d_4           [256]  [128, 256, 846]      512.0   
5_embeddings.ReLU_5                      -  [128, 256, 846]          -   
6_embeddings.Conv1d_6        [256, 512, 3]  [128, 512, 846]   393.728k   
7_embeddings.BatchNorm1d_7           [512]  [128, 512, 846]     1.024k   
8_embeddings.ReLU_8                      -  [128, 512, 846]          -   
9_lstm  

In [14]:
torch.cuda.empty_cache()
asr.train()
asr.save_model()

if asr.log: wandb.finish()

latest_model_name = asr.model_name

--------------------------------------------------------------------------------
				TRAINING
--------------------------------------------------------------------------------


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: nefario7 (use `wandb login --relogin` to force relogin)



Epoch- 1


Epoch 1/105 | Train Loss 3.5916 | Learning Rate 0.0020
Time lapsed = 0:05:03.446878
Time left = 8:45:58.475312
Saving Model!
Saving Checkpoint!

Epoch- 2


Epoch 2/105 | Train Loss 2.8856 | Learning Rate 0.0020
Time lapsed = 0:10:07.427872
Time left = 8:41:22.535408
Saving Checkpoint!

Epoch- 3


...

Epoch 104/105 | Train Loss 0.0073 | Learning Rate 0.0000
Time lapsed = 8:47:44.881506
Time left = 0:05:04.470014
Saving Checkpoint!

Epoch- 105


Epoch 105/105 | Train Loss 0.0072 | Learning Rate 0.0000
Time lapsed = 8:52:49.047834
Time left = 0:00:00
Saving Model!
Saving Checkpoint!

Epoch- 106


Epoch 106/105 | Train Loss 0.0072 | Learning Rate 0.0000
Time lapsed = 8:57:52.887909
Time left = -1 day, 23:54:55.538793
Saving Checkpoint!

Epoch- 107


Train:  18%|█████████████████████                                                                                                   | 39/222 [00:54<04:04,  1.33s/it, loss=0.0070, lr=0.0000]

KeyboardInterrupt: 

# Testing

In [ ]:
# # def calculate_levenshtein(h, y, lh, ly, decoder):
# #     # TODO: call the decoder's decode method and get beam_results and out_len (Read the docs about the decode method's outputs)
# #     # Input to the decode method will be h and its lengths lh 
# #     # You need to pass lh for the 'seq_lens' parameter. This is not explicitly mentioned in the git repo of ctcdecode.

# #     batch_size = 128
# #     dist = 0
# #     phoneme_map = dict(zip(PHONEMES, PHONEME_MAP))
# #     beam_results, beam_scores, timesteps, out_lens = decoder.decode(h)

# #     for i in range(batch_size): 
# #         h_sliced = beam_results[i][0][:out_lens[i][0]]
# #         h_string = ''.join([phoneme_map[PHONEMES[idx]] for idx in h_sliced])

# #         y_sliced = y[i][:ly[i]]
# #         y_string = ''.join([phoneme_map[PHONEMES[idx]] for idx in y_sliced])
        
# #         dist += Levenshtein.distance(h_string, y_string)

# #     dist/=batch_size
# #     print(dist)

# #     # return dist

# config = {
#     '': '',
#     'batch_size': 128,
#     'epochs': 1,
#     'scheduler': None,              # CosineAnnealingLR, ReduceLRonPlateau
#     'optimizer': 'Adam',            # SGD, Adam, AdamW
#     'optim': {'lr': 0.002},
#     'decoder': {'beam_width': 5, 'cutoff_top_n': 40, 'cutoff_prob': 1.0},
#     'arch': {'input_size': 13, 'embedding_size': [64, 128], 'hidden_size': 256, 'num_layers': 4},
#     'save': True,
#     'log': True,
#     'randomize': False,
# }
# torch.cuda.empty_cache()
# decoder = CTCBeamDecoder(
#         PHONEMES,
#         alpha=0,
#         beta=0,
#         cutoff_top_n=40,
#         cutoff_prob=1.0,
#         beam_width=5,
#         blank_id=0,
#         log_probs_input=True
# )
# model = SpeechNet(config).cuda()
# optimizer = optim.Adam(model.parameters(), lr=0.002)
# criterion = nn.CTCLoss().cuda()

# for i, data in enumerate(train_loader):
#     torch.cuda.empty_cache()
#     x, y, x_len, y_len = data
#     print("Iteration ", i, "-"*50)
#     print("Input = ", x.shape, x_len.shape)
#     print("Target = ", y.shape, y_len.shape)

#     model.train()
#     optimizer.zero_grad()

#     x = x.cuda()
#     y = y.cuda()
#     output, output_len = model(x.cuda(), x_len)
#     # print(output[0], output_len[0])
#     # print(y[0], y_len[0])
#     print("Output = ", output.shape, output_len.shape)
#     loss = criterion(torch.transpose(output, 0, 1), y, output_len, y_len)
#     print("Training Loss =", loss)

#     # output = torch.transpose(output, 0, 1)
#     # print("Output Transposed = ", output.shape)
#     ld = calculate_levenshtein(decoder, output, y, output_len, y_len)
#     print(ld)
#     loss.backward()
#     optimizer.step()

#     # Write a test code do perform a single forward pass and also compute the Levenshtein distance
#     # Make sure that you are able to get this right before going on to the actual training
#     # You may encounter a lot of shape errors
#     # Printing out the shapes will help in debugging
#     # Keep in mind that the Loss which you will use requires the input to be in a different format and the decoder expects it in a different format
#     # Make sure to read the corresponding docs about it
# del model
# # beam_results, beam_scores, timesteps, out_lens = decoder.decode(output)
# # print("Decoder = ", beam_results.shape, beam_scores.shape, timesteps.shape, out_lens.shape)
# # phoneme_map = dict(zip(PHONEMES, PHONEME_MAP))
# # # First datapoint in batch
# # test_beam = beam_results[0]
# # test_len = out_lens[0]

# # print(test_beam.shape)
# # print(test_len.shape)

# # # First Beam
# # h_sliced = test_beam[0][:test_len[0]]
# # print("Top beam = ", h_sliced.shape)
# # h_string = ''.join([phoneme_map[PHONEMES[idx]] for idx in h_sliced])
# # print("Top beam mapping = ", h_string)
# # print("Top beam length = ", len(h_string))

# # print("\n")

# # # Target Output
# # print(y.shape)
# # print(y_len.shape)
# # y_sliced = y[0][:y_len[0]]
# # print("Target Seq = ", y_sliced.shape)
# # y_string = ''.join([phoneme_map[PHONEMES[idx]] for idx in y_sliced])
# # print("Target Seq mapping = ", y_string)
# # print("Target Seq length = ", len(y_string))

# # # Score
# # print("\n\nLevenshtein Distance = ", Levenshtein.distance(h_string, y_string))

In [ ]:
# print(output[0][0].sum())

tensor(1.0000, device='cuda:0', grad_fn=<SumBackward0>)


# Submission

In [ ]:
class SpeechInference():
    def __init__(self, bw=30):
        self.phonemes = PHONEMES
        self.phoneme_map = dict(zip(PHONEMES, PHONEME_MAP))

        self.drive_dir = r'/content/cmudrive/IDL'
        # Dataset and dataloader
        self.test_data = LibriSamples(partition='test')
        self.test_loader = DataLoader(
            self.test_data, 
            batch_size=256, 
            shuffle=False, 
            drop_last=False, 
            collate_fn = self.test_data.collate_fn
            )
        
        self.decoder = CTCBeamDecoder(
                PHONEME_MAP,
                blank_id=0,
                log_probs_input=True,
                beam_width=bw, 
                cutoff_top_n= 40,
                cutoff_prob= 1.0
        )
        
    def get_predictions(self, model):
        model.eval()
        predictions = []
        batch_bar = tqdm(total=len(self.test_loader), position=0, leave=False, desc='Test')
        with torch.no_grad():
            for i, data in tqdm(enumerate(self.test_loader)):
                x, x_len = data
                x = x.cuda()

                with torch.no_grad():    
                    output, output_len = model(x, x_len)

                beam_results, beam_scores, timesteps, out_lens = self.decoder.decode(output, seq_lens=output_len)
                for b in range(output.shape[0]):
                    predictions.append(convert_to_string(beam_results[b][0], out_lens[b][0], self.phoneme_map, self.phonemes))
                
                batch_bar.update()
            batch_bar.close()

        return predictions

    def load_model(self, model_name, model_type): 
        meta_path = os.path.join(self.drive_dir,  model_type, model_name, 'model_config.yaml')
        with open(meta_path, 'r') as meta:
            args = yaml.safe_load(meta)

        model_path = os.path.join(self.drive_dir, model_type, model_name, 'model.pth')
        model = SpeechNet(args).cuda()
        model.load_state_dict(torch.load(model_path))
        return model, args

    def simple_inference(self, model_name, model_type):
        print("Running inference...")
        self.timestamp = datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S')
        model, args = self.load_model(model_name, model_type)
        preds = self.get_predictions(model)
        
        return preds

    def ensemble_inference(self, model_names, model_type):
        print("Running ensembled inference...")
        self.timestamp = datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S')

        prelim_labels = []
        for name in model_names:
            print("\n\n\tModel : ", name)
            model, args = self.load_model(name, model_type)
            prelim_labels.append(self.__get_labels(model, args))

        accs = [86.146, 85.79, 84.95]
        w = accs / np.sum(accs)

        print("Combining predictions...")
        labels_df = pd.DataFrame(prelim_labels)
        labels_df = labels_df.transpose()
        ensembled_labels = labels_df.mode(axis=1, dropna=False).iloc[:, 0].tolist()
        # ensembled_labels = np.where((df.iloc[:,1] == df.iloc[:, 2]), df.iloc[:, 1], df.iloc[:, 0]).tolist()

        return labels_df, ensembled_labels

    def generate_submission(self, save_path, preds): 
        print("Generating Submission CSV...")
        sub_dir = os.path.join(self.drive_dir, save_path + self.timestamp)
        try:
            os.mkdir(sub_dir)
        except:
            print("Couldn't create folder for submission.csv")
            
        sub_path = os.path.join(sub_dir, 'submission.csv')

        with open(sub_path, 'w') as f:
            csvwrite = csv.writer(f)
            csvwrite.writerow(['id', 'predictions'])
            for i in range(len(preds)):
                csvwrite.writerow([i, preds[i]])

        print(f"File saved at : {sub_path}")
        return sub_path

In [ ]:
model_name = 'L4512-Conv2-3L-ReLU-512-4-0.4_b64_e125_sCALR_oAdamP_rLSTM_lr0.002'
# model_name = latest_model_name
# models = [
#           r'full_Finale_b16384_e25_c16_bnafter_wNone_oadamp_sNone_d0.25_lr0.001-ver81',
#           r'full_Ensemble1_b16384_e20_c16_bnafter_wNone_oadamp_sNone_d0.25_lr0.001-ver48',
#           r'full_Ensemble2_b16384_e15_c32_bnafter_wNone_oadamp_sNone_d0.25_lr0.001-ver81'
#           ]

model_type = r'hw3-final'
sub_path = r'hw3-submission/'
inference = SpeechInference(bw=40)

predictions = inference.simple_inference(model_name, model_type)
submission_path = inference.generate_submission(sub_path, predictions)

# Simple and Ensemble Inference
# labels = inference.simple_inference(model_name, model_type)
# labels_df, labels = inference.ensemble_inference(models, model_type)

print(f"Preview of submission.csv")
df = pd.read_csv(submission_path)
df.head(10)

Data: 100%|██████████| 2620/2620 [00:01<00:00, 2466.12it/s]


Running inference...


Test: 100%|██████████| 11/11 [02:43<00:00, 12.72s/it]
11it [02:43, 14.88s/it]


Generating Submission CSV...
File saved at : /content/cmudrive/IDL/hw3-submission/2022-04-13_16-20-47/submission.csv
Preview of submission.csv


,id,predictions
0,0,.HIbIgAnhkhnf?UzdkhmplEnthgenstDhWizrd.HUHAdvA...
1,1,.kivnatsO.rnisthmyndthDIzmhmrIzcyld.
2,2,.hgOldhnfoRchnhndhHApIlyf.
3,3,.HIWhzlykhpthmyfaDrinhWE.DAnd?etWhznatmyfaDr.
4,4,..olsODeRWhzhstRipliNpEj.HUtrnditthmd.
5,5,.DisWhzsOsWIthlEdIsr.AndinshmmAnrydUTiNkSIdyd.
6,6,.bhtDenDhpikcrWhzgonezkWiklIezhkEm..
7,7,..sistrnOdU?UHIRDIzmaRvhlz.
8,8,.tEk?UoRplEshndlethsIWhtDhkRisphlkhnSOth?U.
9,9,.lykiznat?hNmAstr.DO.yAmhnOldmAn..


In [ ]:
print(submission_path)
! kaggle competitions submit -c automatic-speech-recognition-slack -f $submission_path -m "Finale 40"

/content/cmudrive/IDL/hw3-submission/2022-04-13_16-20-47/submission.csv
100% 215k/215k [00:05<00:00, 43.9kB/s]
Successfully submitted to Automatic Speech Recognition (Slack)